- Utilisez ces méthodes dans le jeu de données wine_quality
- Observer la pertinence des labels affectés
- Entrainer un modèle de classification avec ces labels affectés.
- il faudra en amont de ces méthodes enlever certains des labels en leur affectant -1

In [76]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.semi_supervised import LabelPropagation, SelfTrainingClassifier, LabelPropagation, LabelSpreading
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, f1_score

In [3]:
url = "https://gist.githubusercontent.com/jsz4n/b7ca11015784086788022a539935d0cf/raw/a8c3abf0a31f5c0df5e0ddd76fb9b289bac9bed1/winequality-red.csv"

In [4]:
df = pd.read_csv(url, sep=";")

In [5]:
df.head()

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  
0      9.4        5  
1      9.8        5  
2      9.8        5  
3      9.8        6  
4      9.4        5

In [10]:
X = df.copy()

In [11]:
y = X.pop("quality")

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2,random_state=42)

## Préparation du mask

In [13]:
mask = np.random.rand(y_train.shape[0])>.75

In [14]:
y_train[mask] = -1

In [16]:
(y_train==-1).sum()

np.int64(307)

## Préparation du pipeline

In [27]:
preprocessing_pipeline = Pipeline( steps = [
    ("normalisation", StandardScaler()),
])
preprocessing_pipeline.set_output(transform="pandas")

Pipeline(steps=[('normalisation', StandardScaler())])

In [28]:
fullpipeline = Pipeline(steps=[
    ("preprocessing", preprocessing_pipeline),
    ("model", RandomForestClassifier())
])

## Entrainement du SelfTrainingClassifier et du LabelPropagation

In [75]:
stc_pipeline = Pipeline( steps=[
    ("preprocessing", preprocessing_pipeline), # on doit normaliser car le SVC fait des distances entre les lignes
    ("stc", SelfTrainingClassifier(SVC(probability=True)))
])
stc_pipeline.fit(X_train,y_train)
y_train_stc_svc  = stc_pipeline.predict(X_train)

In [78]:
stc_dt = SelfTrainingClassifier(DecisionTreeClassifier())
stc_dt.fit(X_train, y_train)
y_train_stc = stc_dt.predict(X_train)

In [53]:
label_p_pipeline = Pipeline( steps=[
    ("preprocessing", preprocessing_pipeline), # on doit normaliser car le LabelPropagation fait des distances entre les lignes
    ("label_propagator",LabelPropagation())
])

In [51]:
label_p_pipeline.fit(X_train, y_train)
y_train_lp = label_p_pipeline.predict(X_train)

/home/jeremie/Documents/Cours/MachineLearningSOPRA/.venv/lib/python3.11/site-packages/sklearn/semi_supervised/_label_propagation.py:230: RuntimeWarning: invalid value encountered in divide
  probabilities /= normalizer


In [52]:
y_train_lp

array([6, 6, 6, ..., 5, 7, 7])

In [85]:
label_s_pipeline = Pipeline( steps=[
    ("preprocessing", preprocessing_pipeline), # on doit normaliser car le LabelSpreading fait des distances entre les lignes
    ("label_spreader",LabelSpreading())
])

In [86]:
label_s_pipeline.fit(X_train, y_train)
y_train_ls = label_s_pipeline.predict(X_train)

/home/jeremie/Documents/Cours/MachineLearningSOPRA/.venv/lib/python3.11/site-packages/sklearn/semi_supervised/_label_propagation.py:230: RuntimeWarning: invalid value encountered in divide
  probabilities /= normalizer


## Recherche des meilleurs hyperparamètres

In [56]:
param_grid = { "model__max_depth":[3,5,7,10], "model__min_samples_leaf":[3,9], "model__n_estimators":[30,50]}

In [87]:
grid_clf = GridSearchCV(estimator=fullpipeline, param_grid=param_grid, n_jobs=4)

In [88]:
grid_clf

GridSearchCV(estimator=Pipeline(steps=[('preprocessing',
                                        Pipeline(steps=[('normalisation',
                                                         StandardScaler())])),
                                       ('model',
                                        RandomForestClassifier(max_depth=10,
                                                               min_samples_leaf=3,
                                                               n_estimators=30))]),
             n_jobs=4,
             param_grid={'model__max_depth': [3, 5, 7, 10],
                         'model__min_samples_leaf': [3, 9],
                         'model__n_estimators': [30, 50]})

In [89]:
grid_clf.fit(X_train, y_train_ls)

GridSearchCV(estimator=Pipeline(steps=[('preprocessing',
                                        Pipeline(steps=[('normalisation',
                                                         StandardScaler())])),
                                       ('model',
                                        RandomForestClassifier(max_depth=10,
                                                               min_samples_leaf=3,
                                                               n_estimators=30))]),
             n_jobs=4,
             param_grid={'model__max_depth': [3, 5, 7, 10],
                         'model__min_samples_leaf': [3, 9],
                         'model__n_estimators': [30, 50]})

## Entrainement du modèle avec les meilleurs hyperparamètres

In [90]:
grid_clf.best_params_

{'model__max_depth': 10,
 'model__min_samples_leaf': 3,
 'model__n_estimators': 50}

In [91]:
fullpipeline.set_params(**grid_clf.best_params_)

Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('normalisation', StandardScaler())])),
                ('model',
                 RandomForestClassifier(max_depth=10, min_samples_leaf=3,
                                        n_estimators=50))])

In [92]:
fullpipeline.fit(X_train, y_train_stc)

Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('normalisation', StandardScaler())])),
                ('model',
                 RandomForestClassifier(max_depth=10, min_samples_leaf=3,
                                        n_estimators=50))])

## Évaluation du modèle

In [93]:
y_pred = fullpipeline.predict(X_test)

In [94]:
accuracy_score(y_test, y_pred)

0.61875

Suite aux tests on remarque que l'on obtient à peu près les mêmes accuracy que avec les 25% de données déja labellisés